In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2

In [2]:
cap = cv2.VideoCapture(0)

In [3]:
from tensorflow.keras.models import load_model

model = load_model("my_model.h5")

In [4]:
def detect_face_mask(img):
    y_pred = model.predict(img.reshape(1,224,224,3))
    return y_pred[0][0]

In [5]:
def draw_label(img, text, pos, bg_color):
    text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, cv2.FILLED)
    end_x = pos[0] + text_size[0][0] + 2
    end_y = pos[1] + text_size[0][1] - 2
    cv2.rectangle(img, pos, (end_x, end_y), bg_color, cv2.FILLED)
    cv2.putText(img, text, pos, cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0),1,cv2.LINE_AA)

In [6]:
haar = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
def detect_face(img):
    cords = haar.detectMultiScale(img)
    return cords

In [7]:
while True:
    ret, frame = cap.read()
    img = cv2.resize(frame, (224, 224))  
    img = img / 255.0                    
    y_pred = model.predict(img.reshape(1, 224, 224, 3), verbose=0) 
    coords = detect_face(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
    for x,y,w,h in coords:
        cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)
    label = int(y_pred[0][0] > 0.5)
    if label == 0:
        draw_label(frame, 'Masked', (30,30),(0,255,0))
    else:
        draw_label(frame, 'No Mask', (30,30),(0,0,255))
    cv2.imshow('window', frame)
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break
cv2.destroyAllWindows()

In [8]:
# while True:
#     ret, frame = cap.read()

#     img = cv2.resize(frame, (224, 224))  
#     img = img / 255.0        
#     img_tensor = tf.convert_to_tensor(img.reshape(1, 224, 224, 3), dtype=tf.float32)
#     y_pred = model(img_tensor, training=False, verbose=False)            
#     # y_pred = model.predict_on_batch(img.reshape(1, 224, 224, 3))
#     # y_pred = model.predict(img.reshape(1, 224, 224, 3)) 

#     label = int(y_pred[0][0] > 0.5)
#     if label == 0:
#         draw_label(frame, 'Masked', (30,30),(0,255,0))
#     else:
#         draw_label(frame, 'No Mask', (30,30),(0,0,255))
#     cv2.imshow('window', frame)
#     if cv2.waitKey(1) & 0xFF == ord('x'):
#         break
# cv2.destroyAllWindows()